In [8]:
import numpy as np
import pandas as pd
import re

In [9]:
df = pd.read_csv('pune_commercial_listings_nobroker.csv')

In [10]:
df.head()

,locality,property_type,location_text,area_sqft,rent_amount,maintenance_amount,deposit_amount,total_monthly_cost,rent_per_sqft,rent_outlier,sqft_outlier,rent_psf_outlier
0,Kothrud,Nan,Hapoy Colony Near Van Devi Mandir Karve Nagar ...,650.0,12000,0,24000,12000,18.46,0,0,0
1,Kothrud,Shop,"Gandhi Bhavan Rd, Gandhi Bhavan (Maharashtra G...",160.0,20000,0,60000,20000,125.00,0,0,0
2,Kothrud,Shop,"Chandani Chowk , Shinde Farm Golden Group,",500.0,100000,0,200000,100000,200.00,0,0,0
3,Kothrud,Office,"Paschimanagri,, City Pride- Kothrud",230.0,100000,13000,500000,113000,434.78,0,0,0
4,Kothrud,Nan,"Late. G A Kulkarni Road, Opposite Karishma Soc...",100.0,10000,0,10000,10000,100.00,0,0,0


### raw_rent column cleaning

In [11]:
# ---------------------------------------------------------
# 1) IMPORT LIBRARIES
# ---------------------------------------------------------
import pandas as pd
import re

# ---------------------------------------------------------
# 2) LOAD ORIGINAL FILE
# ---------------------------------------------------------
df = pd.read_csv("pune_commercial_listings_nobroker.csv")
print("Loaded rows:", len(df))

# ---------------------------------------------------------
# 3) HELPER FUNCTION TO PARSE RUPEE VALUES
# ---------------------------------------------------------
def parse_amount_rupee_unit(amount_str, unit_str):
    if not amount_str:
        return None
    
    try:
        val = float(amount_str.replace(",", "").strip())
    except:
        return None
    
    unit = (unit_str or "").lower()
    
    if "lac" in unit or "lakh" in unit:
        return int(val * 100000)
    if unit in ("k",):
        return int(val * 1000)
    
    return int(val)

# ---------------------------------------------------------
# 4) EXTRACT MAINTENANCE INR
# ---------------------------------------------------------
def extract_maintenance_inr(text):
    if pd.isna(text):
        return None
    
    t = str(text)
    
    # No maintenance case
    if "No Extra Maintenance" in t:
        return 0
    
    # Find all ₹ amounts
    matches = re.findall(
        r"₹\s*([\d.,]+)\s*(Lacs?|Lac|lakh|lakhs|K|k)?",
        t,
        flags=re.I
    )
    
    # If 2 amounts present → maintenance is the second one
    if len(matches) >= 2:
        amt_str, unit_str = matches[1]
        return parse_amount_rupee_unit(amt_str, unit_str)
    
    # Fallback
    return 0

# APPLY TO DATAFRAME
df["Maintenance_inr"] = df["Rent_raw"].apply(extract_maintenance_inr)
print("Added Maintenance_inr.")

# ---------------------------------------------------------
# 5) CALCULATE TOTAL MONTHLY COST
# ---------------------------------------------------------
df["Total_monthly_cost"] = df["Rent_inr"] + df["Maintenance_inr"]
print("Added Total_monthly_cost.")

# ---------------------------------------------------------
# 6) SAVE BACK TO THE SAME FILE
# ---------------------------------------------------------
df.to_csv(
    "pune_commercial_listings_nobroker.csv",
    index=False,
    encoding="utf-8-sig"
)

print("File updated successfully → pune_commercial_listings_nobroker.csv")


Loaded rows: 650


KeyError: 'Rent_raw'

In [ ]:
df.head()

,Locality,Title,Location,Rent_raw,Rent_inr,Deposit_raw,Sqft_raw,Sqft,Price_per_sqft,Parking,Furnishing,Available_from,Property_type,Detail_URL,Maintenance_inr,Total_monthly_cost
0,Kothrud,"Co-Working space in The Business Hub, Pune for...",hapoy colony near van devi mandir karve nagar ...,"₹ 12,000 No Extra Maintenance",12000,"₹ 24,000",650 sqft,650,18.46,Parking,Furnished,22-10-2025,NaN,https://www.nobroker.in/property/commercial/re...,0,12000
1,Kothrud,"Shop in Kothrud, Pune for Rent","Gandhi Bhavan Rd, Gandhi Bhavan (Maharashtra G...","₹ 20,000 No Extra Maintenance",20000,"₹ 60,000",160 sqft,160,125.00,Parking,Unfurnished,18-11-2025,Shop,https://www.nobroker.in/property/commercial/re...,0,20000
2,Kothrud,"Shop in Kothrud, Pune for Rent","Chandani chowk , Shinde Farm Golden Group,",₹ 1 Lac No Extra Maintenance,100000,₹ 2 Lacs,500 sqft,500,66.67,Parking,Furnished,10-11-2025,Shop,https://www.nobroker.in/property/commercial/re...,0,100000
3,Kothrud,"Office Space in Kothrud, Pune for Rent","Paschimanagri,, City Pride- Kothrud","₹ 1 Lac + ₹ 13,000 Maintenance",100000,₹ 5 Lacs,230 sqft,230,81.30,Parking,Furnished,14-11-2025,Office Space,https://www.nobroker.in/property/commercial/re...,13000,113000
4,Kothrud,"Co-Working space in Lookwell Salon - Kothrud, ...","Late. G A kulkarni road, Opposite Karishma Soc...","₹ 10,000 No Extra Maintenance",10000,"₹ 10,000",100 sqft,100,2.44,Parking,Furnished,28-10-2025,NaN,https://www.nobroker.in/property/commercial/re...,0,10000


## Cleaning Deposit Column

In [ ]:
# ---------------------------------------------------------
# 1) IMPORTS
# ---------------------------------------------------------
import pandas as pd
import re

# ---------------------------------------------------------
# 2) LOAD ORIGINAL FILE
#    (make sure it's CLOSED in Excel / other apps)
# ---------------------------------------------------------
file_path = "pune_commercial_listings_nobroker.csv"
df = pd.read_csv(file_path)
print("Loaded rows:", len(df))

# Quick peek
print(df["Deposit_raw"].head(10))


# ---------------------------------------------------------
# 3) HELPER: parse deposit amount with units
# ---------------------------------------------------------
def parse_deposit_amount(text):
    """
    Convert '₹ 24,000', '₹ 2 Lacs', '₹ 1.82 Crores' etc. into integer rupees.
    """
    if pd.isna(text):
        return None

    t = str(text).strip()

    # Optional: handle "No Deposit" style, just in case
    if "no deposit" in t.lower():
        return 0

    # Regex: ₹ [number] [optional unit]
    m = re.search(
        r"₹\s*([\d.,]+)\s*(Lacs?|Lac|lakh|lakhs|Crores?|Crore|Cr|cr)?",
        t,
        flags=re.I,
    )

    if not m:
        return None

    amount_str = m.group(1)
    unit_str = (m.group(2) or "").lower()

    # Clean amount
    try:
        val = float(amount_str.replace(",", "").strip())
    except Exception:
        return None

    # Scale by unit
    if any(u in unit_str for u in ["lac", "lakh"]):
        val = val * 100000        # 1 lac = 1,00,000
    elif any(u in unit_str for u in ["crore", "cr"]):
        val = val * 10000000       # 1 crore = 1,00,00,000

    return int(val)


# ---------------------------------------------------------
# 4) CREATE NUMERIC Deposit_inr COLUMN
# ---------------------------------------------------------
df["Deposit_inr"] = df["Deposit_raw"].apply(parse_deposit_amount)

print("\nSample after cleaning deposit:")
print(df[["Deposit_raw", "Deposit_inr"]].head(20))


# ---------------------------------------------------------
# 5) SAVE BACK TO THE SAME FILE
# ---------------------------------------------------------
df.to_csv(file_path, index=False, encoding="utf-8-sig")
print(f"\nUpdated file saved → {file_path}")


Loaded rows: 650
0      ₹ 24,000
1      ₹ 60,000
2      ₹ 2 Lacs
3      ₹ 5 Lacs
4      ₹ 10,000
5      ₹ 50,000
6      ₹ 3 Lacs
7    ₹ 8.1 Lacs
8      ₹ 35,000
9    ₹ 7.2 Lacs
Name: Deposit_raw, dtype: object

Sample after cleaning deposit:
    Deposit_raw  Deposit_inr
0      ₹ 24,000        24000
1      ₹ 60,000        60000
2      ₹ 2 Lacs       200000
3      ₹ 5 Lacs       500000
4      ₹ 10,000        10000
5      ₹ 50,000        50000
6      ₹ 3 Lacs       300000
7    ₹ 8.1 Lacs       810000
8      ₹ 35,000        35000
9    ₹ 7.2 Lacs       720000
10  ₹ 1.65 Lacs       165000
11     ₹ 2 Lacs       200000
12     ₹ 65,000        65000
13   ₹ 4.5 Lacs       450000
14     ₹ 75,000        75000
15    ₹ 75 Lacs      7500000
16     ₹ 50,000        50000
17      ₹ 1 Lac       100000
18   ₹ 2.5 Lacs       250000
19     ₹ 40,000        40000

Updated file saved → pune_commercial_listings_nobroker.csv


### Cleaning available from, sqft Parking

In [ ]:
# ---------------------------------------------------------
# 1) IMPORTS
# ---------------------------------------------------------
import pandas as pd
import numpy as np
import re

# ---------------------------------------------------------
# 2) LOAD ORIGINAL FILE
#    (make sure it's CLOSED in Excel / other apps)
# ---------------------------------------------------------
file_path = "pune_commercial_listings_nobroker.csv"
df = pd.read_csv(file_path)
print("Loaded rows:", len(df))
print("Columns:", df.columns.tolist())


# ---------------------------------------------------------
# 3) CLEAN Sqft_raw / Sqft
#    - Parse numeric sqft from strings like "650 sqft"
#    - Turn "000 sqft" (0) into NaN
# ---------------------------------------------------------
def parse_sqft(text):
    if pd.isna(text):
        return None
    # just grab the number part
    m = re.search(r"([\d,]+(?:\.\d+)?)", str(text))
    if not m:
        return None
    try:
        return float(m.group(1).replace(",", ""))
    except Exception:
        return None

df["Sqft"] = df["Sqft_raw"].apply(parse_sqft)

# Treat zero as missing (comes from "000 sqft")
df.loc[df["Sqft"] == 0, "Sqft"] = np.nan

print("\nSqft cleaned. Sample:")
print(df[["Sqft_raw", "Sqft"]].head(10))


# ---------------------------------------------------------
# 4) STANDARDIZE Furnishing
#    Values observed: 'Furnished', 'Semi Furnished', 'Unfurnished'
#    Normalize to: Furnished / Semi-Furnished / Unfurnished
# ---------------------------------------------------------
df["Furnishing"] = df["Furnishing"].astype(str).str.strip()

furnish_map = {
    "Furnished": "Furnished",
    "Semi Furnished": "Semi-Furnished",
    "Semi-Furnished": "Semi-Furnished",
    "Unfurnished": "Unfurnished",
}

df["Furnishing"] = df["Furnishing"].replace(furnish_map)

print("\nFurnishing value counts:")
print(df["Furnishing"].value_counts())


# ---------------------------------------------------------
# 5) CLEAN Parking
#    Currently everything is just 'Parking' (label, not info).
#    We'll set it to NaN so it doesn't mislead analysis.
# ---------------------------------------------------------
df["Parking"] = df["Parking"].astype(str).str.strip()
df.loc[df["Parking"] == "Parking", "Parking"] = np.nan

print("\nUnique Parking values after cleaning:", df["Parking"].unique())


# ---------------------------------------------------------
# 6) STANDARDIZE Available_from
#    - Current format: DD-MM-YYYY (string)
#    - Convert to datetime, then back to ISO string YYYY-MM-DD
# ---------------------------------------------------------
df["Available_from"] = df["Available_from"].astype(str).str.strip()

# parse dates
df["Available_from_date"] = pd.to_datetime(
    df["Available_from"],
    format="%d-%m-%Y",
    errors="coerce"
)

# back to ISO string, keep in the original column
df["Available_from"] = df["Available_from_date"].dt.strftime("%Y-%m-%d")

print("\nAvailable_from sample after cleaning:")
print(df[["Available_from"]].head(10))


# ---------------------------------------------------------
# 7) TRIM BASIC TEXT COLUMNS (nice to have)
# ---------------------------------------------------------
for col in ["Locality", "Title", "Location", "Property_type"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

print("\nSample Title / Location after trimming:")
print(df[["Locality", "Title"]].head(5))


# ---------------------------------------------------------
# 8) SAVE BACK TO THE SAME FILE (OVERWRITE)
# ---------------------------------------------------------
df.to_csv(file_path, index=False, encoding="utf-8-sig")
print(f"\nFile cleaned and saved → {file_path}")


Loaded rows: 650
Columns: ['Locality', 'Title', 'Location', 'Rent_raw', 'Rent_inr', 'Deposit_raw', 'Sqft_raw', 'Sqft', 'Price_per_sqft', 'Parking', 'Furnishing', 'Available_from', 'Property_type', 'Detail_URL', 'Maintenance_inr', 'Total_monthly_cost', 'Deposit_inr']

Sqft cleaned. Sample:
   Sqft_raw   Sqft
0  650 sqft  650.0
1  160 sqft  160.0
2  500 sqft  500.0
3  230 sqft  230.0
4  100 sqft  100.0
5  200 sqft  200.0
6  111 sqft  111.0
7  850 sqft  850.0
8  350 sqft  350.0
9  430 sqft  430.0

Furnishing value counts:
Furnishing
Unfurnished       382
Furnished         141
Semi-Furnished    127
Name: count, dtype: int64

Unique Parking values after cleaning: [nan]

Available_from sample after cleaning:
  Available_from
0     2025-10-22
1     2025-11-18
2     2025-11-10
3     2025-11-14
4     2025-10-28
5     2025-10-26
6     2025-10-24
7     2023-01-31
8     2025-10-14
9     2025-09-26

Sample Title / Location after trimming:
  Locality                                              Titl

In [ ]:
import pandas as pd

file_path = "pune_commercial_listings_nobroker.csv"

# Load file
df = pd.read_csv(file_path)
print("Before drop:", df.columns.tolist())

# Columns to drop
cols_to_drop = [
    "Rent_raw",
    "Deposit_raw",
    "Sqft_raw",
    "Parking",
    "Available_from",    # ONLY raw version; cleaned is stored in Available_from_date or ISO col
    "Detail_URL"
]

# Drop only if column actually exists
existing_cols = [c for c in cols_to_drop if c in df.columns]
df = df.drop(columns=existing_cols)

print("\nAfter drop:", df.columns.tolist())

# Save back to the same file
df.to_csv(file_path, index=False, encoding="utf-8-sig")

print(f"\nUpdated file saved → {file_path}")


Before drop: ['Locality', 'Title', 'Location', 'Rent_raw', 'Rent_inr', 'Deposit_raw', 'Sqft_raw', 'Sqft', 'Price_per_sqft', 'Parking', 'Furnishing', 'Available_from', 'Property_type', 'Detail_URL', 'Maintenance_inr', 'Total_monthly_cost', 'Deposit_inr', 'Available_from_date']

After drop: ['Locality', 'Title', 'Location', 'Rent_inr', 'Sqft', 'Price_per_sqft', 'Furnishing', 'Property_type', 'Maintenance_inr', 'Total_monthly_cost', 'Deposit_inr', 'Available_from_date']

Updated file saved → pune_commercial_listings_nobroker.csv


In [ ]:
import pandas as pd
import numpy as np
import re

file_path = "pune_commercial_listings_nobroker.csv"

df = pd.read_csv(file_path)
print("Loaded:", len(df), "rows")

# ================================================================
# 🔥 STEP 1 — REMOVE DUPLICATES (based on Detail_URL)
# ================================================================
if "Detail_URL" in df.columns:
    before = len(df)
    df = df.drop_duplicates(subset=["Detail_URL"], keep="first")
    after = len(df)
    print(f"Duplicates removed: {before - after}")
else:
    print("Detail_URL column already dropped earlier.")


# ================================================================
# 🔥 STEP 2 — RECOMPUTE Price_per_sqft PROPERLY
# ================================================================
def safe_div(a, b):
    try:
        if b and b > 0:
            return round(a / b, 2)
        return np.nan
    except:
        return np.nan

df["Price_per_sqft"] = df.apply(
    lambda x: safe_div(x["Rent_inr"], x["Sqft"]),
    axis=1
)

# ================================================================
# 🔥 STEP 3 — ADD Rent_per_sqft (same as price per sqft)
# ================================================================
df["Rent_per_sqft"] = df["Price_per_sqft"]  # alias for clarity


# ================================================================
# 🔥 STEP 4 — CLEAN Property_type
# ================================================================
df["Property_type"] = df["Property_type"].astype(str).str.strip().str.title()

map_ptype = {
    "Office": "Office",
    "Office Space": "Office",
    "Commercial": "Commercial Space",
    "Commercial Space": "Commercial Space",
    "Retail": "Retail",
    "Showroom": "Showroom",
    "Warehouse": "Warehouse",
    "Restaurant": "Restaurant",
}

df["Property_type_clean"] = df["Property_type"].replace(map_ptype)


# ================================================================
# 🔥 STEP 5 — NORMALIZE Location text
# ================================================================
def clean_location(text):
    if pd.isna(text):
        return None
    t = str(text)
    t = t.replace("\n", " ").replace("\r", " ")
    t = re.sub(r"\s+", " ", t)  # multiple spaces → single
    return t.strip().title()    # uniform formatting

df["Location_clean"] = df["Location"].apply(clean_location)


# ================================================================
# 🔥 STEP 6 — OUTLIER DETECTION
# ================================================================

# --- Rent outliers ---
df["Rent_outlier"] = df["Rent_inr"].apply(
    lambda x: 1 if (x is not None and (x < 3000 or x > 3000000)) else 0
)

# --- Sqft outliers ---
df["Sqft_outlier"] = df["Sqft"].apply(
    lambda x: 1 if (pd.notna(x) and (x < 50 or x > 20000)) else 0
)

# --- Rent_per_sqft outliers ---
df["RPSF_outlier"] = df["Rent_per_sqft"].apply(
    lambda x: 1 if (pd.notna(x) and (x < 5 or x > 500)) else 0
)


# ================================================================
# ✔ Final: Save cleaned dataset
# ================================================================
df.to_csv(file_path, index=False, encoding="utf-8-sig")

print("All cleaning steps completed successfully!")
print("Saved →", file_path)
print(df.head())


Loaded: 650 rows
Detail_URL column already dropped earlier.
All cleaning steps completed successfully!
Saved → pune_commercial_listings_nobroker.csv
  Locality                                              Title  \
0  Kothrud  Co-Working space in The Business Hub, Pune for...   
1  Kothrud                     Shop in Kothrud, Pune for Rent   
2  Kothrud                     Shop in Kothrud, Pune for Rent   
3  Kothrud             Office Space in Kothrud, Pune for Rent   
4  Kothrud  Co-Working space in Lookwell Salon - Kothrud, ...   

                                            Location  Rent_inr   Sqft  \
0  hapoy colony near van devi mandir karve nagar ...     12000  650.0   
1  Gandhi Bhavan Rd, Gandhi Bhavan (Maharashtra G...     20000  160.0   
2         Chandani chowk , Shinde Farm Golden Group,    100000  500.0   
3                Paschimanagri,, City Pride- Kothrud    100000  230.0   
4  Late. G A kulkarni road, Opposite Karishma Soc...     10000  100.0   

   Price_per_sqft   F

In [ ]:
import pandas as pd
import numpy as np
import re

file_path = "pune_commercial_listings_nobroker.csv"

# 1) LOAD
df = pd.read_csv(file_path)
print("Loaded rows:", len(df))
print("Columns:", df.columns.tolist())


# 2) ENSURE NUMERIC TYPES
num_cols = [
    "Rent_inr",
    "Maintenance_inr",
    "Total_monthly_cost",
    "Deposit_inr",
    "Sqft",
    "Price_per_sqft",    # will be recomputed but make sure numeric
    "Rent_per_sqft"      # will be recomputed
]

for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# 3) RECOMPUTE PRICE_PER_SQFT + RENT_PER_SQFT
def safe_div(a, b):
    try:
        if pd.notna(a) and pd.notna(b) and b > 0:
            return round(a / b, 2)
    except Exception:
        pass
    return np.nan

if "Rent_inr" in df.columns and "Sqft" in df.columns:
    df["Price_per_sqft"] = df.apply(
        lambda row: safe_div(row["Rent_inr"], row["Sqft"]), axis=1
    )
    df["Rent_per_sqft"] = df["Price_per_sqft"]

# 4) CLEAN PROPERTY_TYPE
if "Property_type" in df.columns:
    df["Property_type"] = df["Property_type"].astype(str).str.strip().str.title()

    map_ptype = {
        "Office": "Office",
        "Office Space": "Office",
        "Commercial": "Commercial Space",
        "Commercial Space": "Commercial Space",
        "Retail": "Retail",
        "Showroom": "Showroom",
        "Warehouse": "Warehouse",
        "Restaurant": "Restaurant",
    }

    df["Property_type_clean"] = df["Property_type"].replace(map_ptype)
else:
    print("Property_type column not found; skipping that cleaning step.")

# 5) NORMALIZE LOCATION TEXT
def clean_location(text):
    if pd.isna(text):
        return None
    t = str(text)
    t = t.replace("\n", " ").replace("\r", " ")
    t = re.sub(r"\s+", " ", t)
    return t.strip().title()

if "Location" in df.columns:
    df["Location_clean"] = df["Location"].apply(clean_location)

# 6) TRIM BASIC TEXT FIELDS
for col in ["Locality", "Title"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# 7) REBUILD OUTLIER FLAGS
# These are simple rule-based flags; adjust thresholds if needed.
if "Rent_inr" in df.columns:
    df["Rent_outlier"] = df["Rent_inr"].apply(
        lambda x: 1 if (pd.notna(x) and (x < 3000 or x > 3000000)) else 0
    )

if "Sqft" in df.columns:
    df["Sqft_outlier"] = df["Sqft"].apply(
        lambda x: 1 if (pd.notna(x) and (x < 50 or x > 20000)) else 0
    )

if "Rent_per_sqft" in df.columns:
    df["RPSF_outlier"] = df["Rent_per_sqft"].apply(
        lambda x: 1 if (pd.notna(x) and (x < 5 or x > 500)) else 0
    )

print("\nOutlier flag counts:")
for col in ["Rent_outlier", "Sqft_outlier", "RPSF_outlier"]:
    if col in df.columns:
        print(col, "→", df[col].value_counts().to_dict())

# 8) SAVE BACK TO SAME FILE
df.to_csv(file_path, index=False, encoding="utf-8-sig")
print("\nFinal cleaned file saved →", file_path)


Loaded rows: 650
Columns: ['Locality', 'Title', 'Location', 'Rent_inr', 'Sqft', 'Price_per_sqft', 'Furnishing', 'Property_type', 'Maintenance_inr', 'Total_monthly_cost', 'Deposit_inr', 'Available_from_date', 'Rent_per_sqft', 'Property_type_clean', 'Location_clean', 'Rent_outlier', 'Sqft_outlier', 'RPSF_outlier']

Outlier flag counts:
Rent_outlier → {0: 650}
Sqft_outlier → {0: 645, 1: 5}
RPSF_outlier → {0: 609, 1: 41}

Final cleaned file saved → pune_commercial_listings_nobroker.csv


In [ ]:
import pandas as pd

file_path = "pune_commercial_listings_nobroker.csv"

df = pd.read_csv(file_path)

# Columns we want to drop
columns_to_drop = [
    "Location",
    "Price_per_sqft",
    "Furnishing",
    "Property_type",
    "Available_from_date"
]

# Drop only if they exist in the file
existing_cols = [c for c in columns_to_drop if c in df.columns]
df = df.drop(columns=existing_cols)

# Save cleaned file
df.to_csv(file_path, index=False, encoding="utf-8-sig")

print("Dropped columns:", existing_cols)
print("Final file saved →", file_path)


Dropped columns: ['Location', 'Price_per_sqft', 'Furnishing', 'Property_type', 'Available_from_date']
Final file saved → pune_commercial_listings_nobroker.csv


In [ ]:
import pandas as pd
import re

file_path = "pune_commercial_listings_nobroker.csv"

df = pd.read_csv(file_path)

# Function to clean Title
def clean_title(title, locality=None):
    if pd.isna(title):
        return title

    t = str(title)

    # Remove variants of "for rent"
    t = re.sub(r'(\bfor rent\b|\bFor Rent\b|\bFOR RENT\b)', '', t, flags=re.I)

    # Remove "Pune" or ", Pune"
    t = re.sub(r',?\s*Pune', '', t, flags=re.I)

    # Remove repeated locality names inside title
    if locality:
        pattern = rf",?\s*{re.escape(locality)}"
        t = re.sub(pattern, '', t, flags=re.I)

    # Remove "India", pin codes if present
    t = re.sub(r'\b\d{6}\b', '', t)
    t = t.replace("India", "")

    # Remove multiple spaces & commas
    t = re.sub(r'\s+', ' ', t).strip()
    t = re.sub(r',$', '', t).strip()

    return t

df["Title"] = df.apply(lambda row: clean_title(row["Title"], row["Locality"]), axis=1)

df.to_csv(file_path, index=False, encoding="utf-8-sig")

print("Title column cleaned successfully!")


Title column cleaned successfully!


In [ ]:
import pandas as pd

file_path = "pune_commercial_listings_nobroker.csv"

df = pd.read_csv(file_path)

rename_map = {
    "Locality": "locality",
    "Title": "listing_title",
    "Rent_inr": "rent_amount",
    "Sqft": "area_sqft",
    "Maintenance_inr": "maintenance_amount",
    "Total_monthly_cost": "total_monthly_cost",
    "Deposit_inr": "deposit_amount",
    "Rent_per_sqft": "rent_per_sqft",
    "Property_type_clean": "property_type",
    "Location_clean": "cleaned_location",
    "Rent_outlier": "rent_outlier",
    "Sqft_outlier": "sqft_outlier",
    "RPSF_outlier": "rent_psf_outlier"
}

df.rename(columns=rename_map, inplace=True)

df.to_csv(file_path, index=False, encoding="utf-8-sig")

print("✔ Column names renamed successfully!")
print(df.columns)


✔ Column names renamed successfully!
Index(['locality', 'listing_title', 'rent_amount', 'area_sqft',
       'maintenance_amount', 'total_monthly_cost', 'deposit_amount',
       'rent_per_sqft', 'property_type', 'cleaned_location', 'rent_outlier',
       'sqft_outlier', 'rent_psf_outlier'],
      dtype='object')


In [ ]:
import pandas as pd

file_path = "pune_commercial_listings_nobroker.csv"

df = pd.read_csv(file_path)

# 1. Rename cleaned_location → location_text
df.rename(columns={"cleaned_location": "location_text"}, inplace=True)

# 2. Reorder columns
final_order = [
    "locality",
    "property_type",
    "listing_title",
    "location_text",
    "area_sqft",
    "rent_amount",
    "maintenance_amount",
    "deposit_amount",
    "total_monthly_cost",
    "rent_per_sqft",
    "rent_outlier",
    "sqft_outlier",
    "rent_psf_outlier"
]

# Only include columns that exist in the DataFrame
existing_cols = [col for col in final_order if col in df.columns]

df = df[existing_cols]

# 3. Save final cleaned file
df.to_csv(file_path, index=False, encoding="utf-8-sig")

print("✔ Columns renamed and reordered successfully!")
print("Final columns:", list(df.columns))


✔ Columns renamed and reordered successfully!
Final columns: ['locality', 'property_type', 'listing_title', 'location_text', 'area_sqft', 'rent_amount', 'maintenance_amount', 'deposit_amount', 'total_monthly_cost', 'rent_per_sqft', 'rent_outlier', 'sqft_outlier', 'rent_psf_outlier']


In [ ]:
import pandas as pd
import re

file_path = "pune_commercial_listings_nobroker.csv"
df = pd.read_csv(file_path)

def normalize_property_type(pt):
    if pd.isna(pt):
        return None
    
    text = pt.lower()

    # Office
    if re.search(r'office', text):
        return "Office"

    # Shop
    if re.search(r'shop', text):
        return "Shop"

    # Showroom / Retail
    if re.search(r'showroom|retail', text):
        return "Showroom"

    # Warehouse / Godown / Storage
    if re.search(r'warehouse|godown|storage', text):
        return "Warehouse"

    # Restaurant / Cafe
    if re.search(r'restaurant|cafe', text):
        return "Restaurant"

    # Commercial generic
    if re.search(r'commercial', text):
        return "Commercial"

    # Default
    return pt.strip().title()

df["property_type"] = df["property_type"].apply(normalize_property_type)

df.to_csv(file_path, index=False, encoding="utf-8-sig")

print("✔ Property types normalized!")
print(df["property_type"].value_counts())


✔ Property types normalized!
property_type
Office        372
Shop          183
Warehouse      44
Commercial     19
Nan            18
Showroom        8
Restaurant      6
Name: count, dtype: int64


In [ ]:
import pandas as pd

file_path = "pune_commercial_listings_nobroker.csv"

df = pd.read_csv(file_path)

# Drop the column
df.drop(columns=["listing_title"], inplace=True)

# Save updated file
df.to_csv(file_path, index=False, encoding="utf-8-sig")

print("✔ Column 'listing_title' dropped successfully!")
print(df.columns)


KeyError: "['listing_title'] not found in axis"

In [12]:
df.head()

,locality,property_type,location_text,area_sqft,rent_amount,maintenance_amount,deposit_amount,total_monthly_cost,rent_per_sqft,rent_outlier,sqft_outlier,rent_psf_outlier
0,Kothrud,Nan,Hapoy Colony Near Van Devi Mandir Karve Nagar ...,650.0,12000,0,24000,12000,18.46,0,0,0
1,Kothrud,Shop,"Gandhi Bhavan Rd, Gandhi Bhavan (Maharashtra G...",160.0,20000,0,60000,20000,125.00,0,0,0
2,Kothrud,Shop,"Chandani Chowk , Shinde Farm Golden Group,",500.0,100000,0,200000,100000,200.00,0,0,0
3,Kothrud,Office,"Paschimanagri,, City Pride- Kothrud",230.0,100000,13000,500000,113000,434.78,0,0,0
4,Kothrud,Nan,"Late. G A Kulkarni Road, Opposite Karishma Soc...",100.0,10000,0,10000,10000,100.00,0,0,0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   locality            650 non-null    object 
 1   property_type       650 non-null    object 
 2   location_text       650 non-null    object 
 3   area_sqft           593 non-null    float64
 4   rent_amount         650 non-null    int64  
 5   maintenance_amount  650 non-null    int64  
 6   deposit_amount      650 non-null    int64  
 7   total_monthly_cost  650 non-null    int64  
 8   rent_per_sqft       593 non-null    float64
 9   rent_outlier        650 non-null    int64  
 10  sqft_outlier        650 non-null    int64  
 11  rent_psf_outlier    650 non-null    int64  
dtypes: float64(2), int64(7), object(3)
memory usage: 61.1+ KB


In [16]:
df.isnull().sum()

locality               0
property_type          0
location_text          0
area_sqft             57
rent_amount            0
maintenance_amount     0
deposit_amount         0
total_monthly_cost     0
rent_per_sqft         57
rent_outlier           0
sqft_outlier           0
rent_psf_outlier       0
dtype: int64

In [18]:
import pandas as pd
import numpy as np

# Load the cleaned dataset
file_name = 'pune_commercial_listings_nobroker.csv'
df = pd.read_csv(file_name)

print(f"--- Loading {file_name} ---")

# --- 1. Store the indices of rows with missing values ---
# We'll need this to recalculate 'rent_per_sqft' later
original_missing_indices = df[df['area_sqft'].isnull()].index
print(f"Found {len(original_missing_indices)} rows with missing 'area_sqft'.")

# --- 2. Impute 'area_sqft' using grouped median ---
print("Imputing 'area_sqft' using median 'area_sqft' from their 'locality'...")
# Step 2a: Calculate median area_sqft for each locality
locality_median_sqft = df.groupby('locality')['area_sqft'].transform('median')
# Step 2b: Fill missing values with the grouped median
df['area_sqft'] = df['area_sqft'].fillna(locality_median_sqft)

# --- 3. Handle potential remaining NaNs (fallback) ---
# This is a safety check in case a locality had *only* NaN values
remaining_nans = df['area_sqft'].isnull().sum()
if remaining_nans > 0:
    print(f"Found {remaining_nans} rows where locality had no median. Filling with global median...")
    global_median_sqft = df['area_sqft'].median()
    df['area_sqft'] = df['area_sqft'].fillna(global_median_sqft)

# --- 4. Recalculate 'rent_per_sqft' ---
print("Recalculating 'rent_per_sqft' for imputed rows...")
# Recalculate only for the rows that were originally missing
df.loc[original_missing_indices, 'rent_per_sqft'] = df.loc[original_missing_indices, 'rent_amount'] / df.loc[original_missing_indices, 'area_sqft']
# Round to 2 decimal places for consistency
df['rent_per_sqft'] = df['rent_per_sqft'].round(2)

# --- 5. Final check ---
print("\n--- Missing values after imputation ---")
print(df.isnull().sum())

# --- 6. Save the new, imputed file ---
imputed_file_name = 'pune_commercial_listings_imputed.csv'
df.to_csv(imputed_file_name, index=False)

print(f"\nImputation complete. Fully cleaned data saved to '{imputed_file_name}'.")

--- Loading pune_commercial_listings_nobroker.csv ---
Found 57 rows with missing 'area_sqft'.
Imputing 'area_sqft' using median 'area_sqft' from their 'locality'...
Recalculating 'rent_per_sqft' for imputed rows...

--- Missing values after imputation ---
locality              0
property_type         0
location_text         0
area_sqft             0
rent_amount           0
maintenance_amount    0
deposit_amount        0
total_monthly_cost    0
rent_per_sqft         0
rent_outlier          0
sqft_outlier          0
rent_psf_outlier      0
dtype: int64

Imputation complete. Fully cleaned data saved to 'pune_commercial_listings_imputed.csv'.


np.int64(25)

In [21]:
df.drop_duplicates()

,locality,property_type,location_text,area_sqft,rent_amount,maintenance_amount,deposit_amount,total_monthly_cost,rent_per_sqft,rent_outlier,sqft_outlier,rent_psf_outlier
0,Kothrud,Nan,Hapoy Colony Near Van Devi Mandir Karve Nagar ...,650.0,12000,0,24000,12000,18.46,0,0,0
1,Kothrud,Shop,"Gandhi Bhavan Rd, Gandhi Bhavan (Maharashtra G...",160.0,20000,0,60000,20000,125.00,0,0,0
2,Kothrud,Shop,"Chandani Chowk , Shinde Farm Golden Group,",500.0,100000,0,200000,100000,200.00,0,0,0
3,Kothrud,Office,"Paschimanagri,, City Pride- Kothrud",230.0,100000,13000,500000,113000,434.78,0,0,0
4,Kothrud,Nan,"Late. G A Kulkarni Road, Opposite Karishma Soc...",100.0,10000,0,10000,10000,100.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
644,Wagholi,Shop,"Nagar Road, Mercedes-Benz Silver Star Workshop",250.0,40000,0,100000,40000,160.00,0,0,0
645,Wagholi,Showroom,"Ubale Nagar, Wagholi Pune, Maharashtra 412207 ...",243.0,225000,0,1350000,225000,925.93,0,0,0
646,Wagholi,Warehouse,"Hxfm+P23, Kalubai Nagar, Raisoni College",243.0,35000,0,100000,35000,144.03,0,0,0
647,Wagholi,Shop,"Kawade Wasti, , Trimurti Gym And Health Club (...",100.0,4500,0,20000,4500,45.00,0,0,0


In [35]:
import pandas as pd

# Define the file you want to clean
input_file = 'pune_commercial_listings_imputed.csv'
output_file = 'pune_commercial_listings_FINAL.csv'

# Load the dataset
df = pd.read_csv(input_file)

print(f"--- Loading {input_file} ---")
print(f"Rows before de-duplication: {len(df)}")
print(f"Duplicates found: {df.duplicated().sum()}")

# Drop the duplicate rows
df_cleaned = df.drop_duplicates()

print(f"\n--- Duplicates removed ---")
print(f"Rows after de-duplication: {len(df_cleaned)}")

# Save the final, cleaned file
df_cleaned.to_csv(output_file, index=False)
print(f"\nFinal, clean file saved as: {output_file}")

--- Loading pune_commercial_listings_imputed.csv ---
Rows before de-duplication: 650
Duplicates found: 25

--- Duplicates removed ---
Rows after de-duplication: 625

Final, clean file saved as: pune_commercial_listings_FINAL.csv


In [36]:
df_cleaned.duplicated().sum()

np.int64(0)

In [1]:
import pandas as pd

df = pd.read_csv('pune_commercial_listings_FINAL.csv')
df_original = df.copy()

print(f"Original: {len(df)} records")

# FIX 1: Remove area < 50 sqft
df_cleaned = df[df['area_sqft'] >= 50].copy()

# FIX 2: Classify NaN properties
def classify_nan_property(row):
    area = row['area_sqft']
    rent_psf = row['rent_per_sqft']
    location = str(row['location_text']).lower()
    rent = row['rent_amount']
    
    if area < 100 and rent_psf > 100:
        return 'Kiosk'
    elif 'cafe' in location or 'restaurant' in location:
        return 'Restaurant'
    elif 'cowork' in location or 'space' in location:
        return 'Office'
    elif 'shop' in location or 'retail' in location:
        return 'Shop'
    elif 100 <= area <= 300 and 50 <= rent_psf <= 200:
        return 'Office'
    elif area > 400 and rent < 50000:
        return 'Warehouse'
    else:
        if rent_psf > 150:
            return 'Office'
        elif rent_psf > 50:
            return 'Shop'
        else:
            return 'Warehouse'

nan_indices = df_cleaned[df_cleaned['property_type'] == 'Nan'].index
for idx in nan_indices:
    df_cleaned.loc[idx, 'property_type'] = classify_nan_property(df_cleaned.loc[idx])

# FIX 3: Outliers already flagged
outliers = df_cleaned[df_cleaned['rent_psf_outlier'] == 1]

# Save
df_cleaned.to_csv('pune_commercial_listings_FINAL.csv', index=False)

print(f"\n✅ CSV Updated: {len(df_cleaned)} records")
print(f"Records: {len(df_original)} → {len(df_cleaned)}")
print(f"Quality: {(len(df_cleaned)/len(df_original)*100):.1f}%")


Original: 625 records

✅ CSV Updated: 621 records
Records: 625 → 621
Quality: 99.4%


In [2]:
df_check = pd.read_csv('pune_commercial_listings_FINAL.csv')

print("✅ VERIFICATION:")
print(f"   Records: {len(df_check)} (should be 621)")
print(f"   NaN types: {len(df_check[df_check['property_type']=='Nan'])} (should be 0)")
print(f"   Area < 50: {len(df_check[df_check['area_sqft']<50])} (should be 0)")

if len(df_check)==621 and len(df_check[df_check['property_type']=='Nan'])==0:
    print("\n✅ ALL CHECKS PASSED! Data is clean!")


✅ VERIFICATION:
   Records: 621 (should be 621)
   NaN types: 0 (should be 0)
   Area < 50: 0 (should be 0)

✅ ALL CHECKS PASSED! Data is clean!


In [ ]:
df.head()